# Comparison Tables By Metrice Scores

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif
from impyute.imputation.cs import fast_knn
import pickle

# Scores
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Ensemble models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# Ignoring Errors
import warnings
warnings.simplefilter('ignore')

In [2]:
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
scoring = make_scorer(roc_auc_score)
def gsearch(classifier, X_train, y_train, param_grid, scoring=scoring):
    gs = GridSearchCV(estimator=classifier,
                      param_grid=param_grid, 
                      scoring=scoring,
                      cv=5,
                      n_jobs=-1)
    gs = gs.fit(X_train, y_train)
    return (gs.best_score_, gs.best_params_, gs.best_estimator_)

def collect_scores(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_prob)
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    
    tpr = tp/(tp + fn)
    tnr = tn/(fp + tn)
    ppv = tp/(tp + fp)
    npv = tn/(tn + fn)
    plr = tpr/(1 - tnr)
    nlr = (1 - tpr)/tnr
    y_test_size = y_test.shape[0]
    fn = fn/y_test_size
    fp = fp/y_test_size
    return [tpr, tnr, fn, fp, ppv, npv, plr, nlr, auc]

In [3]:
def best_estimator(X, y):
    best_lr, best_svm, best_rf = [0, 0], [0, 0], [0, 0]

    pipe_lr = Pipeline([['sc', StandardScaler()], ['clf', LogisticRegression(random_state=1)]])
    pipe_svm = Pipeline([['sc', StandardScaler()], ['clf', SVC(probability=True, random_state=1)]])
    rf = RandomForestClassifier(random_state=1, n_jobs=-1)

    lr_grid = [{'clf__C': param_range,
            'clf__penalty': ['l1','l2']}]

    svm_grid = [{'clf__C': param_range,
                'clf__kernel': ['rbf','sigmoid']}]
    
    rf_grid = [{
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]}]

    kf = StratifiedKFold(10, random_state=1)
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        lr = gsearch(pipe_lr, X_train, y_train, lr_grid)
        svm = gsearch(pipe_svm, X_train, y_train, svm_grid)
        rf_best = gsearch(rf, X_train, y_train, rf_grid)
        if lr[0] > best_lr[0]:
            best_lr[0] = lr[0]
            best_lr[1] = lr
        if svm[0] > best_svm[0]: 
            best_svm[0] = svm[0]
            best_svm[1] = svm
        if rf_best[0] > best_rf[0]: 
            best_rf[0] = rf_best[0]
            best_rf[1] = rf_best
            
    return (best_lr[1][2], best_svm[1][2], best_rf[1][2], (best_lr[1][1], best_svm[1][1], best_rf[1][1]))

# Importing Data

In [4]:
data = pd.read_excel("NAFLD_en.xlsx")
features_df = data.iloc[:,1:-9]
targets_df = data[['Significant Fibrosis (No=0, Yes=1) (If Fibrosis 2 and above, there is Significant Fibrosis)',
                         'Advanced Fibrosis (No=0, Yes=1) (If Fibrosis is 3 and above, there is Advanced Fibrosis)']]

In [5]:
missing_val_counts = {}
for col in features_df.columns:
    missing_val_counts[col] = features_df[col].isnull().sum()

In [6]:
baseline_features = []
for col, value in missing_val_counts.items():
    if value == 0:
        baseline_features.append(col)
baseline_features = features_df[baseline_features]
dropped_baseline = baseline_features.drop(['Steatosis', 'Activity'], 1)

## Baseline (No missed values)

In [7]:
# models_4_SF = best_estimator(dropped_baseline.values, targets_df.iloc[:,0].values)
# models_4_AF = best_estimator(dropped_baseline.values, targets_df.iloc[:,1].values)

In [8]:
with open('best_models.db', 'rb') as models:
    keeping_models = pickle.load(models)
    
    models_4_SF = keeping_models['0']['SF']
    models_4_AF = keeping_models['0']['AF']

In [9]:
models_4_AF[3] # best parameters of the models

({'clf__C': 0.001, 'clf__penalty': 'l2'},
 {'clf__C': 100.0, 'clf__kernel': 'rbf'},
 {'bootstrap': True,
  'max_depth': 80,
  'max_features': 3,
  'min_samples_leaf': 4,
  'min_samples_split': 8,
  'n_estimators': 1000})

In [10]:
lr_best_sf, svm_best_sf, rf_best_sf = models_4_SF[0], models_4_SF[1], models_4_SF[2]
lr_best_af, svm_best_af, rf_best_af = models_4_AF[0], models_4_AF[1], models_4_AF[2]

In [11]:
bagging = BaggingClassifier(DecisionTreeClassifier(), n_jobs=-1, random_state=1)

sf_models = [lr_best_sf, svm_best_sf, rf_best_sf, bagging]
af_models = [lr_best_af, svm_best_af, rf_best_af, bagging]

In [12]:
comparison_table = {}
kf = StratifiedKFold(10, random_state=1)
X = dropped_baseline.values

for idx, models in enumerate([sf_models, af_models]):
    if idx == 0: target = "SF"
    else: target = "AF"
    comparison_table.setdefault(target, [])
    for mdl in models:
        avrg = []
        y = targets_df.iloc[:,idx].values
        for train_index, test_index in kf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            avrg.append(collect_scores(mdl, X_train, X_test, y_train, y_test))
        avrg = pd.DataFrame(np.array(avrg)).replace({np.inf:np.nan}).values
        avrg_list = np.round(np.nanmean(avrg, axis=0, dtype=np.float64),3).tolist()
        avrg_list[0], avrg_list[1] = round(avrg_list[0]*100), round(avrg_list[1]*100)
        avrg_list[2], avrg_list[3] = avrg_list[2], avrg_list[3]
        comparison_table[target].append(avrg_list)

In [13]:
columns = ['Sensitivity (%)', 'Specificity (%)', 'FN', 'FP', 'PPV', 'NPV', 'PLR', 'NLR', 'AUC']
index = ['LR', 'SVM', 'RandomForest', 'Bagging']
index1 = pd.MultiIndex.from_tuples([('SIGNIFICANT', 'LR'), ('FIBROSIS', 'SVM'), ('','RandomForest'), ('', 'Bagging')])
sf_table_df = pd.DataFrame(np.array(comparison_table['SF']), index=index1, columns=columns)
sf_table_df

Sensitivity (%)  Specificity (%)     FN     FP  \
SIGNIFICANT LR                       48.0             82.0  0.188  0.117   
FIBROSIS    SVM                      44.0             84.0  0.203  0.101   
            RandomForest             38.0             88.0  0.223  0.079   
            Bagging                  42.0             78.0  0.208  0.140   

                            PPV    NPV    PLR    NLR    AUC  
SIGNIFICANT LR            0.592  0.736  3.139  0.648  0.705  
FIBROSIS    SVM           0.614  0.726  3.186  0.670  0.711  
            RandomForest  0.648  0.715  4.886  0.710  0.700  
            Bagging       0.532  0.704  2.528  0.760  0.644

In [14]:
index2 = pd.MultiIndex.from_tuples([('ADVANCED', 'LR'), ('FIBROSIS', 'SVM'), ('', 'RandomForest'), ('', 'Bagging')])
af_table_df = pd.DataFrame(np.array(comparison_table['AF']), index=index2, columns=columns)
af_table_df

Sensitivity (%)  Specificity (%)     FN     FP    PPV  \
ADVANCED LR                       32.0             90.0  0.121  0.081  0.467   
FIBROSIS SVM                      32.0             88.0  0.121  0.101  0.386   
         RandomForest              6.0            100.0  0.169  0.002  0.833   
         Bagging                  19.0             95.0  0.144  0.038  0.543   

                         NPV    PLR    NLR    AUC  
ADVANCED LR            0.861  3.437  0.750  0.754  
FIBROSIS SVM           0.857  3.366  0.769  0.703  
         RandomForest  0.830  0.000  0.947  0.765  
         Bagging       0.845  5.499  0.846  0.712

## 25% Threshold

In [15]:
features_df = features_df.drop(['Steatosis', 'Activity'], 1)
missing_data_percentages = features_df.isnull().mean()
threshold_25 = [col for col, persantage in missing_data_percentages.items() if persantage < 0.25]
threshold_25_df = features_df[threshold_25]

knn_25 = fast_knn(threshold_25_df.values, k=30)

In [16]:
# models_25_SF = best_estimator(knn_25, targets_df.iloc[:,0].values)
# models_25_AF = best_estimator(knn_25, targets_df.iloc[:,1].values)

In [17]:
with open('best_models.db', 'rb') as models:
    keeping_models = pickle.load(models)
    
    models_25_SF = keeping_models['25']['SF']
    models_25_AF = keeping_models['25']['AF']

In [18]:
models_25_AF[3] # best parameters of the models

({'clf__C': 0.001, 'clf__penalty': 'l2'},
 {'clf__C': 100.0, 'clf__kernel': 'sigmoid'},
 {'bootstrap': True,
  'max_depth': 80,
  'max_features': 3,
  'min_samples_leaf': 3,
  'min_samples_split': 8,
  'n_estimators': 200})

In [19]:
lr_best_sf_25, svm_best_sf_25, rf_best_sf_25 = models_25_SF[0], models_25_SF[1], models_25_SF[2]
lr_best_af_25, svm_best_af_25, rf_best_af_25 = models_25_AF[0], models_25_AF[1], models_25_AF[2]

In [20]:
bagging = BaggingClassifier(DecisionTreeClassifier(), n_jobs=-1, random_state=1)

sf_25_models = [lr_best_sf_25, svm_best_sf_25, rf_best_sf_25, bagging]
af_25_models = [lr_best_af_25, svm_best_af_25, rf_best_af_25, bagging]

In [21]:
comparison_table_25 = {}
kf = StratifiedKFold(10, random_state=1)
X = knn_25

for idx, models in enumerate([sf_25_models, af_25_models]):
    if idx == 0: target = "SF"
    else: target = "AF"
    comparison_table_25.setdefault(target, [])
    for mdl in models:
        avrg = []
        y = targets_df.iloc[:,idx].values
        for train_index, test_index in kf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            avrg.append(collect_scores(mdl, X_train, X_test, y_train, y_test))
        avrg = pd.DataFrame(np.array(avrg)).replace({np.inf:np.nan}).values
        avrg_list = np.round(np.nanmean(avrg, axis=0, dtype=np.float64),3).tolist()
        avrg_list[0], avrg_list[1] = round(avrg_list[0]*100), round(avrg_list[1]*100)
        avrg_list[2], avrg_list[3] = avrg_list[2], avrg_list[3]
        comparison_table_25[target].append(avrg_list)

In [22]:
index3 = pd.MultiIndex.from_tuples([('SIGNIFICANT', 'LR'), ('FIBROSIS', 'SVM'), ('', 'RandomForest'), ('', 'Bagging')])
sf_table_25_df = pd.DataFrame(np.array(comparison_table_25['SF']), index=index3, columns=columns)
sf_table_25_df

Sensitivity (%)  Specificity (%)     FN     FP  \
SIGNIFICANT LR                       49.0             80.0  0.183  0.127   
FIBROSIS    SVM                      46.0             80.0  0.195  0.129   
            RandomForest             29.0             91.0  0.256  0.060   
            Bagging                  42.0             85.0  0.208  0.094   

                            PPV    NPV    PLR    NLR    AUC  
SIGNIFICANT LR            0.574  0.738  3.569  0.648  0.716  
FIBROSIS    SVM           0.567  0.724  3.446  0.691  0.693  
            RandomForest  0.650  0.694  4.789  0.788  0.707  
            Bagging       0.640  0.723  5.483  0.684  0.706

In [23]:
index4 = pd.MultiIndex.from_tuples([('ADVANCED', 'LR'), ('FIBROSIS', 'SVM'), ('', 'RandomForest'), ('', 'Bagging')])
af_table_25_df = pd.DataFrame(np.array(comparison_table_25['AF']), index=index4, columns=columns)
af_table_25_df

Sensitivity (%)  Specificity (%)     FN     FP    PPV  \
ADVANCED LR                       54.0             83.0  0.082  0.137  0.427   
FIBROSIS SVM                      29.0             85.0  0.127  0.126  0.296   
         RandomForest              7.0            100.0  0.167  0.002  0.667   
         Bagging                  16.0             95.0  0.150  0.038  0.374   

                         NPV    PLR    NLR    AUC  
ADVANCED LR            0.894  3.920  0.552  0.783  
FIBROSIS SVM           0.846  2.089  0.840  0.663  
         RandomForest  0.831  0.000  0.936  0.786  
         Bagging       0.840  4.239  0.879  0.672

## Keeping Models In The Memory

In [24]:
# keep_models = {'0':{'SF':models_4_SF, 'AF':models_4_AF},
#                '25':{'SF':models_25_SF, 'AF':models_25_AF}}
# with open('best_models.db','wb') as models:
#     pickle.dump(keep_models, models)

In [25]:
# with open('models.db', 'rb') as models:
#     keeping_models = pickle.load(models)

In [26]:
# from pandas.plotting import table

# tables = [sf_table_df, af_table_df, sf_table_25_df, af_table_25_df]
# table_names = ['sf_table.png', 'af_table.png', 'sf_25_table.png', 'af_25_table.png']
# for tbl, name in zip(tables, table_names):
#     ax = plt.subplot(111, frame_on=False) # no visible frame
#     ax.xaxis.set_visible(False)  # hide the x axis
#     ax.yaxis.set_visible(False)  # hide the y axis

#     table(ax, tbl)
#     plt.savefig(name, dpi=600, bbox_inches='tight')